In [ ]:
%load_ext aiida
%aiida

In [ ]:
# General imports.
from copy import deepcopy
import numpy as np
import re
import time
from datetime import datetime,timedelta
import pandas as pd
import ipywidgets as ipw
from IPython.display import clear_output,Markdown

from aiida.engine import ExitCode, ToContext, WorkChain, calcfunction,workfunction

In [ ]:
style = {'description_width': '160px'}
layout = {'width': '40%'}
output = ipw.Output()
output1 = ipw.Output()
output2 = ipw.Output()
add_person = ipw.Button(description='Add person')
exit_person = ipw.Button(description='Person left')
people_html = ipw.HTML()
report_html = ipw.HTML()
cash_html = ipw.HTML()
correction_html = ipw.HTML()
info_html = ipw.HTML()

name = ipw.Text(description='Name or nickname',style=style, layout=layout)
email = ipw.Text(description='e-mail',style=style, layout=layout)
startdate = ipw.DatePicker(description='Starting from:',value=datetime.now(),style=style, layout=layout)

In [ ]:
html_phead = """
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:2px;font-family:Arial, sans-serif;font-size:14px;
    overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
    font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-dark{background-color:#c0c0c0;border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-llyw{background-color:#efefef;border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:middle}
</style>
<table class="tg">
<thead>
<tr>"""    
html_ptail =     "</tr></thead><tbody>"
html_report_head = html_phead + "<th class='tg-dark' >Date </th> <th class='tg-dark'>Description</th>" + html_ptail
#
html_people_head = html_phead + "<th class='tg-dark' >Who </th>"
html_people_head += "<th class='tg-dark' >Balance (CHF) </th>"
html_people_head += "<th class='tg-dark' >Coffee (CHF) </th>"
html_people_head += "<th class='tg-dark' >Other (CHF) </th>"
html_people_head += "<th class='tg-dark' >Cash (CHF) </th>"
html_people_head += "<th class='tg-dark' >Cups </th>"
html_people_head += "<th class='tg-dark' >Days member </th>"
html_people_head += "<th class='tg-dark' >Member since </th>"
html_people_head += "<th class='tg-dark' >Left on </th>"
html_people_head += html_ptail
#
tclass = ["", "tg-dark", "tg-llyw"]   

In [ ]:
# input widgets

In [ ]:
event = ipw.Dropdown(options=[],
                     description='Possible actions',style=style, layout=layout)
amount = ipw.FloatText(description='CHF',style=style, layout=layout)
kgcoffee = ipw.FloatText(description='kg',style=style, layout=layout,value =1)
cups_per_day = ipw.FloatText(description='cups/day',style=style, layout=layout,value =1.0)
datei_widget = ipw.DatePicker(description='On:',value=datetime.now(),style=style, layout=layout)
datef_widget = ipw.DatePicker(description='Till:',value=datetime.now(),style=style, layout=layout)
description = ipw.Text(description='Description',style=style, layout=layout)
who = ipw.Dropdown(options=[],description='Person',style=style, layout=layout)
apply = ipw.Button(description='Apply')

In [ ]:
# validate e-mail

def validemail(email):
    pat = "^[a-zA-Z0-9-_]+\.[a-zA-Z0-9-_]+@[a-zA-Z0-9]+\.[a-z]{1,3}$"
    if re.match(pat,email):
        return True
    return False

In [ ]:
def timeconversion(intime):
    if isinstance(intime,float):
        return datetime.fromtimestamp(intime).strftime("%Y-%m-%d")
    else:
        return time.mktime(intime.timetuple())

In [ ]:
def daysactive(person):
    if 'left' in person.extras:
        return np.busday_count(timeconversion(person['started']),timeconversion(person.extras['left']))
    else:
        return np.busday_count(timeconversion(person['started']),datetime.now().strftime("%Y-%m-%d"))

In [ ]:
def BDdays_from_increment(datei,increment): #(float,int) -> (datetime,datetime)
    return pd.to_datetime(timeconversion(datei)),pd.to_datetime(timeconversion(datei)) + pd.offsets.BDay(increment)
def date_in_between(datei,datef,dates):
    for date in dates:
        if datei <= date <= datef:
            return True
    return False
def range_is_valid(datei,datef,person):
    # check wrt person left already done, here we check if a range was already entered for cups consumption
    global days_renormalization
    #days_renormalization[entry['person']]['drenorm'].append([BDdays_from_increment(entry['datei'],entry['amount']),entry['Range_cups/day']])
    datei = pd.to_datetime(timeconversion(datei))
    datef = pd.to_datetime(timeconversion(datef))
    for event in days_renormalization[person['name']]['drenorm']:
        start_date,end_date = event[0]
        if date_in_between(start_date,end_date,[datei,datef]):
            return (
                f"The previous range for cups consumption {start_date.strftime('%Y-%m-%d')} - "
                f"{end_date.strftime('%Y-%m-%d')}\n"
                f"overlaps with the new range {datei.strftime('%Y-%m-%d')} - {datef.strftime('%Y-%m-%d')}\n"
                "nothing done"
                )                
    return None

def date_is_valid(date,person):
    if 'left' in person.extras:
        dateleft = person.extras['left']
    else:
        dateleft = float(3000000000)
    return person['started'] - 100000 < date < dateleft + 100000
def daterange(start_date, end_date):
    """Generate a range of dates between two dates."""
    for n in range((end_date - start_date).days + 1):
        yield start_date + timedelta(n)

def business_days(dates):
    """Filter out weekends and return business days only."""
    return [date for date in dates if date.weekday() < 5]

def get_non_overlapping_busyness_days(list1, list2):
    list1_dates = set()
    list2_dates = set()
    
    # Convert list1 ranges into a set of all dates
    for start, end in list1:
        list1_dates.update(daterange(start, end))
    
    # Convert list2 ranges into a set of all dates
    for start, end in list2:
        list2_dates.update(daterange(start, end))
    
    # Subtract list1 dates from list2 dates to get non-overlapping dates
    non_overlapping_dates = list2_dates - list1_dates
    
    # Filter only business days (Mon-Fri)
    business_days_only = business_days(non_overlapping_dates)
    
    return len(business_days_only)

In [ ]:
# perosn balance computed as:
# personal[person]['coffee']+personal[person]['other']+personal[person]['cash'])-((total_cost - cash)*personal[person]['cups'])/total_cups
# personal is a dictionary with keys the names of the people and values a dictionary with keys 'coffee','other','cash','cups','days','coeff','since','left'
#all_people list with nodes of people
def check_people():
    people_who_left=[]
    global cash,days_renormalization
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_member']}})
    all_people=[]
    emails = []
    # this is also needed for ne actions to check if tehy overlap with previous ones
    days_renormalization = {}
    for node in qb.all(flat=True):
        all_people.append(node)
        days_renormalization[node['name']] = {'drenorm':[],'absences':[],'global':1}
        emails.append(node['email'])
    who.options = sorted([(member['name'],member) for member in all_people if 'left' not in member.extras], key=lambda x: x[0])
    qb = QueryBuilder()
    qb.append(CalcFunctionNode, filters={
        'label': {'in': ['new_event']}})   
    ordered_entries =  sorted([node.outputs.result.get_dict() for node in qb.all(flat=True)], key=lambda d: d['datei'], reverse=True)
    for entry in ordered_entries:
        if 'Range_cups/day' in entry['event']:
            days_renormalization[entry['person']]['drenorm'].append([BDdays_from_increment(entry['datei'],entry['amount']),entry['Range_cups/day']])
        if 'Absence' in entry['event']:
            days_renormalization[entry['person']]['absences'].append(BDdays_from_increment(entry['datei'],entry['amount']))
        if 'default_cups/day' in entry['event']:
            days_renormalization[entry['person']]['global'] = entry['Range_cups/day']
    
    personal = {}
    # sum of effective days for all members. Absences and period renormalizations will be considered later
    # total_day = estimated cups consumed by 
    total_cups = 0
    for person in all_people:
        coeff = days_renormalization[person['name']]['global']
        days = daysactive(person)
        cups = days*coeff
        total_cups += cups
        left=''
        if 'left' in person.extras:
            left = timeconversion(person.extras['left'])
            people_who_left.append(person['name'])
        personal[person['name']] = {'coffee':0,'other':0, 'cash':0, 'cups': cups, 'days':days, 'coeff':coeff, 'since':timeconversion(person['started']), 'left':left}
    cash = 0
    html_report = html_report_head
    html_people = html_people_head
    total_cost=0
    odd = -1
    
    # adjust values according to events
    for entry in ordered_entries:
        what = entry['event']
        if 'cash' in entry['event']:
            cash += entry['amount']
            if 'Donated' not in what:
                personal[entry['person']]['cash'] += entry['amount']
        elif 'Bought' in entry['event']:
            total_cost += entry['amount']
            if 'coffee' in entry['event']:
                personal[entry['person']]['coffee'] += entry['amount']
            else:
                personal[entry['person']]['other'] += entry['amount']
        elif 'Absence' in entry['event']:
            tosubtract = entry['amount']*personal[entry['person']]['coeff']
            personal[entry['person']]['cups'] -= tosubtract
            total_cups -= tosubtract
        elif 'Range_cups/day' in entry['event']:
            range_renorm =   [BDdays_from_increment(entry['datei'],entry['amount'])]
            absences = [absence for absence in days_renormalization[entry['person']]['absences']]
            days_affected = get_non_overlapping_busyness_days(absences, range_renorm)
            tosubtract = days_affected*personal[entry['person']]['coeff']
            personal[entry['person']]['cups'] -= tosubtract
            total_cups -= tosubtract
            toadd = days_affected*entry['Range_cups/day']
            personal[entry['person']]['cups'] += toadd
            total_cups += toadd
        
        html_report += "<tr>"
        html_report += f"<td class={tclass[odd]}> {timeconversion(entry['datei'])} </td>"
        html_report += f"<td class={tclass[odd]}> {entry['description']} </td>"
        html_report += "</tr>" 
        odd *= -1   
     
    #short_report = [(person,(personal[person]['coffee']+personal[person]['other']+personal[person]['cash'])-((total_cost - cash)*personal[person]['cups'])/total_cups,personal[person]['coffee'],personal[person]['other'],personal[person]['cash'],personal[person]['cups'],personal[person]['days'],personal[person]['since'],personal[person]['left']) for person in personal]
    # first estimate of cost balance, will have to be corrected for people who left
    short_report = [
    (
        person,
        (
            personal[person]['coffee']
            + personal[person]['other']
            + personal[person]['cash']
            - ((total_cost - cash) * personal[person]['cups']) / total_cups
        ),
        personal[person]['coffee'],
        personal[person]['other'],
        personal[person]['cash'],
        personal[person]['cups'],
        personal[person]['days'],
        personal[person]['since'],
        personal[person]['left'],
    )
    for person in personal
]

    # correct for people who left
    cost_correction = 0 # positive means a person left positive money
    for person in short_report:
        if person[0] in people_who_left:
            cost_correction += person[1]
            
    # recompute balance
    short_report = [
    (
        person,
        0
        if person in people_who_left
        else (
            personal[person]['coffee']
            + personal[person]['other']
            + personal[person]['cash']
            - ((total_cost - cost_correction - cash) * personal[person]['cups']) / total_cups
        ),
        personal[person]['coffee'],
        personal[person]['other'],
        personal[person]['cash'],
        personal[person]['cups'],
        personal[person]['days'],
        personal[person]['since'],
        personal[person]['left'],
    )
    for person in personal
]

            
    short_report = sorted(short_report, key=lambda d: d[1], reverse=True)
    for person in short_report:
        html_people += "<tr>"
        html_people += f"<td class={tclass[odd]}> {person[0]} </td>"
        html_people += f"<td class={tclass[odd]}> {person[1]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[2]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[3]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[4]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[5]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[6]:.2f} </td>"
        html_people += f"<td class={tclass[odd]}> {person[7]} </td>"
        html_people += f"<td class={tclass[odd]}> {person[8]} </td>"
        html_people += "</tr>" 
        odd *= -1
    people_html.value = html_people
    report_html.value = html_report
    cash_html.value = f"<h3>Available cash: {cash:.2f} CHF</h3>"
    correction_html.value = f"<h3>Cash correction for self inflation: {cost_correction:.2f} CHF</h3>"
    return emails,cash

In [ ]:
display(ipw.VBox([cash_html,correction_html]))

# Persons

In [ ]:
display(ipw.VBox([people_html,startdate,name,email, ipw.HBox([add_person,exit_person]),output,]))

In [ ]:
# Instructions

In [ ]:
# Define the instructions as a formatted string
instructions_text = """
**Select a person and an action. Try to keep your balance (CHF) close to 0.**

Your estimated number of cups  depends on the nomber of working days since your subscription 
multiplied by your consumption coefficinets.
The total number of cups among all members together with the total amount of costs registered 
(cash is accounted only if refunded to someone) is normalized
with respect to each member's #cups. Changmenets to your coefficients affect all balances.

**You can enter money:**
- For stuff you bought (e.g., coffee)
- For cash you add to the system (Twint money to +41 76 53 25 330 (Carlo))
- For donation (donations are not credited!!)
- To request expenses refund from the available cash (ask Carlo)

**Personal cups consumption (1 cup = 1 double espresso):**
- Set your default cups/day consumption (`default_cups/day`) 
- Set a specific cup/day for a date range (`range_cups/day`)
- Specify a period of absence
"""
# Create a styled HTML widget for the description

styled_description = ipw.HTML(
    value="<b><span style='font-size:20px;'>Instructions</span></b>"
)

# Create a Checkbox widget
instructions_checkbox = ipw.Checkbox(
    value=False,
)

# Create an Output widget to control the display of the instructions
output = ipw.Output()

# Function to show or hide instructions based on checkbox state
def on_checkbox_change(change):
    with output:
        output.clear_output()  # Clear previous output
        if change['new']:  # If checkbox is checked, display instructions
            display(Markdown(instructions_text))

# Link the checkbox to the function
instructions_checkbox.observe(on_checkbox_change, names='value')

# Display the checkbox and the output area
display(styled_description,instructions_checkbox, output)

In [ ]:
@calcfunction
def new_person(name,email,startdate):
    return Dict({'name':name.value,'email':email.value,'started':startdate.value})
    
def on_add_person_clicked(b):
    global emails,cash
    with output:
        clear_output()
        if validemail(email.value) and startdate.value is not None and name.value != '':
            if email.value in emails:
                print("person already present")
            else:
                new = new_person(Str(name.value),Str(email.value),Float(timeconversion(startdate.value)))
                new.label = 'nanotech@coffee_member'
                new.set_extra('coeff',1)
                print("Added",new['name'],'starting from: ',timeconversion(new['started']),'pk:',new.pk )
                emails.append(email.value)
    emails,cash = check_people()
    
def on_exit_person_clicked(b):
    global emails,cash,cost_correction
    with output:
        clear_output()
        qb = QueryBuilder()
        qb.append(Node, filters={
            'label': {'in': ['nanotech@coffee_member']}})
        for node in qb.all(flat=True):
            if node['name'] == name.value:
                node.set_extra('left', Float(timeconversion(startdate.value)))
                print("Person",node['name'],'left on: ',timeconversion(node.extras['left']))
    emails,cash,cost_correction = check_people()
            
add_person.on_click(on_add_person_clicked)
exit_person.on_click(on_exit_person_clicked)

# Actions

In [ ]:
@workfunction
def create_action(action):
    action.label = 'nanotech@coffee_action'
    return action

@calcfunction
def new_event(person=None,cups=None,datei=None,datef=None,event=None,description=None,amount=None,kgcoffee=None,available_cash=None):
        
    if 'Bought' in event.value or 'Added' in event.value:
        theamount = np.abs(amount.value)
        if 'coffee' in event.value:
            return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                        'description': f"{person['name']} bought {kgcoffee.value}kg coffee ({description.value}) for {theamount} CHF" })
        else:    
            return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                            'description': f"{person['name']} {event.value} ({description.value}) for {theamount} CHF" })
    elif event.value == 'Donated cash':
        theamount = np.abs(amount.value)
        return Dict({'person': person['name'], 'datei':datei.value, 'amount':theamount, 'event':event.value,
                        'description': f"{person['name']} donated {theamount} CHF ({description.value}) to the common fund" })
    elif event.value == 'Requested cash':
        theamount = np.abs(amount.value)
        if amount.value <= available_cash.value +0.01 :
            return Dict({'person': person['name'], 'datei':datei.value, 'amount': -1.0*theamount, 'event':event.value,
                            'description': f"{person['name']} received {theamount} CHF from the common fund" })
        else:
            return Dict({'person': person['name'], 'datei':datei.value,'amount':0.0,'event':event.value,
                            'description': f"{person['name']} requested {theamount} CHF from the common fund but they are not available. They will not be counted"})
    elif event.value == 'Absence':
        theamount = np.busday_count(timeconversion(datei.value),timeconversion(datef.value))
        return Dict({'person': person['name'], 'datei':datei.value,'amount':theamount,'event':event.value,
                        'description': f"{person['name']}  entered absence from {timeconversion(datei.value)}  until {timeconversion(datef.value)} ({theamount} days {description.value})" })
    elif event.value == 'Range_cups/day':
        theamount = np.busday_count(timeconversion(datei.value),timeconversion(datef.value))
        return Dict({'person': person['name'], 'datei':datei.value,'amount':theamount, 'Range_cups/day':cups.value,'event':event.value,
                        'description': f"{person['name']}  set avg cups/day from {timeconversion(datei.value)}  until {timeconversion(datef.value)} as {cups.value} cups this affects {theamount} days ({description.value})" })
    elif event.value == 'default_cups/day':
        theamount = 0
        person.set_extra('coeff',cups.value)
        return Dict({'person': person['name'], 'datei':datei.value,'amount':theamount,'Range_cups/day':cups.value,'event':event.value,
                        'description': f"{person['name']}  set default cups/day as {cups.value} cups ({description.value})" })

    

In [ ]:
def on_apply_clicked(b):
    global emails, cash
    with output2:
        clear_output()
        person = who.value
        cups=0.0
        kg = 0.0
        datei = timeconversion(datei_widget.value)
        datef = datei
        no_go = False
        if 'Absence' in event.value.value:
            datef = timeconversion(datef_widget.value)            
        if 'coffee' in event.value.value:
            kg = kgcoffee.value
        if 'Range_cups/day' in event.value.value:
            cups = cups_per_day.value
            datef = timeconversion(datef_widget.value)
            check_range = range_is_valid(datei,datef,person)
            if check_range is not None:
                print(check_range)
                no_go = True
        if 'default_cups/day' in event.value.value:
            cups = cups_per_day.value
            datei = datei = timeconversion(datetime.now())
            datef=datei
        if not (date_is_valid(datei,person) and date_is_valid(datef,person)):
            no_go = True
            print("Date is not valid for this person")
        if datef < datei:
            no_go = True
            print("End date is before start date")
        if not no_go:        
            what = new_event(
                person=person,
                datei=Float(datei),
                cups=Float(cups),
                datef=Float(datef),
                event=event.value,
                description=Str(description.value),
                amount=Float(amount.value),
                kgcoffee=Float(kg),
                available_cash=Float(cash)
                )
            print("Added event: ",what['description'])              
        emails,cash,cost_correction = check_people() 
apply.on_click(on_apply_clicked)

def on_event_change(change):
    with output1:
        clear_output()
        if 'Absence' in change['new'].value:
            todisplay = [who,datei_widget,datef_widget,description,info_html]
            description.value =''
            kgcoffee.value=0
            amount.value=0
            info_html.value='Enter the range of dates and a description of the absence'
        elif 'Range_cups/day' in change['new'].value:
            todisplay = [who,datei_widget,datef_widget,description,cups_per_day,info_html]
            description.value =''
            kgcoffee.value=0
            amount.value=0
            info_html.value='Specify the range of dates and the number of cups/day (1 coup = 1 double espresso) to apply for that period. CAnnot overlap with previously specified ranges'   
        elif 'default_cups/day' in change['new'].value:
            todisplay = [who,description,cups_per_day,info_html]
            description.value =''
            kgcoffee.value=0
            amount.value=0
            info_html.value='1 coup = 1 double espresso. This coefficient will be applyed to all days since your subscriptions until now. Except for periods of absence or specific range of cups/day'          
        elif 'Bought coffee' == change['new'].value:
            todisplay = [who,datei_widget,kgcoffee,description,amount,info_html]
            kgcoffee.value=2
            description.value='Konstanz: mondo verde'
            amount.value=60
            info_html.value='Enter the amount of coffee in kg and the cost in CHF'
        else:
            description.value=''
            amount.value=0
            kgcoffee.value=0
            todisplay = [who,datei_widget,description,amount,info_html]
            info_html.value='Check the Instructions box to get instructions'
            
        display(ipw.VBox(todisplay))
        
event.observe(on_event_change,names='value')

In [ ]:
qb = QueryBuilder()
qb.append(Node, filters={
    'label': {'in': ['nanotech@coffee_action']}})
needed_actions=['Bought coffee','default_cups/day','Range_cups/day','Bought accessory','Bought cleaning stuff','Donated cash', 'Added cash','Requested cash','Absence']
existing_actions = {}    
for node in qb.all(flat=True):
    existing_actions[node.value]=node
    #print(node.value,node.pk)
for action in needed_actions:
    if action not in existing_actions:
        existing_actions[action]=create_action(Str(action))
event.options = list(existing_actions.items())


In [ ]:
emails,cash,cost_correction = check_people()
cash_html.value = f"<h3>Available cash: {cash:.2f} CHF</h3>"
correction_html.value = f"<h3>Cash correction for self inflation: {cost_correction:.2f} CHF</h3>"
display(ipw.VBox([event,output1,apply,output2]))

# Report

In [ ]:
display(report_html)

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_member']}})
    for node in qb.all(flat=True):
        print(node.pk,node.get_dict())

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['nanotech@coffee_action']}})
    for node in qb.all(flat=True):
        print(node.pk, node.value)

In [ ]:
if False :
    qb = QueryBuilder()
    qb.append(Node, filters={
        'label': {'in': ['new_event']}})
    for node in qb.all(flat=True):
        print(node.pk, node.outputs.result['event'])

In [ ]:
#timeconversion(float(3000000000 - 100000))